In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [ ]:
df = pd.read_csv('pokemon.csv')
df.columns


Index(['Number', 'Name', 'Type_1', 'Type_2', 'Total', 'HP', 'Attack',
       'Defense', 'Sp_Atk', 'Sp_Def', 'Speed', 'Generation', 'isLegendary',
       'Color', 'hasGender', 'Pr_Male', 'Egg_Group_1', 'Egg_Group_2',
       'hasMegaEvolution', 'Height_m', 'Weight_kg', 'Catch_Rate',
       'Body_Style'],
      dtype='object')

In [ ]:
df = df[['isLegendary','Generation', 'Type_1', 'Type_2', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed','Color','Egg_Group_1','Height_m','Weight_kg','Body_Style']]


0      False
1      False
2      False
3      False
4      False
       ...  
716     True
717     True
718     True
719     True
720     True
Name: isLegendary, Length: 721, dtype: bool

In [ ]:
df['isLegendary'] = df['isLegendary'].astype(int)

In [ ]:
def dummy_creation(df, dummy_categories):
    for i in dummy_categories:
        df_dummy = pd.get_dummies(df[i])
        df = pd.concat([df,df_dummy],axis=1)
        df = df.drop(i, axis=1)
    return(df)

In [ ]:
df = dummy_creation(df, ['Egg_Group_1', 'Body_Style', 'Color','Type_1', 'Type_2'])


In [ ]:
def train_test_splitter(DataFrame, column):
    df_train = DataFrame.loc[df[column] != 1]
    df_test = DataFrame.loc[df[column] == 1]

    df_train = df_train.drop(column, axis=1)
    df_test = df_test.drop(column, axis=1)

    return(df_train, df_test)

df_train, df_test = train_test_splitter(df, 'Generation')

In [ ]:
def label_delineator(df_train, df_test, label):
    
    train_data = df_train.drop(label, axis=1).values
    train_labels = df_train[label].values
    test_data = df_test.drop(label,axis=1).values
    test_labels = df_test[label].values
    return(train_data, train_labels, test_data, test_labels)

train_data, train_labels, test_data, test_labels = label_delineator(df_train, df_test, 'isLegendary')


In [ ]:
def data_normalizer(train_data, test_data):
    train_data = preprocessing.MinMaxScaler().fit_transform(train_data)
    test_data = preprocessing.MinMaxScaler().fit_transform(test_data)
    return(train_data, test_data)

train_data, test_data = data_normalizer(train_data, test_data)

In [ ]:
length = train_data.shape[1]

model = keras.Sequential()
model.add(keras.layers.Dense(500, activation='relu', input_shape=[length,]))
model.add(keras.layers.Dense(2, activation='softmax'))

In [ ]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, train_labels, epochs=400)




Epoch 1/400
18/18 [==============================] - 0s 2ms/step - loss: 0.5659 - accuracy: 0.8912
Epoch 2/400
18/18 [==============================] - 0s 1ms/step - loss: 0.4315 - accuracy: 0.9263
Epoch 3/400
18/18 [==============================] - 0s 1ms/step - loss: 0.3638 - accuracy: 0.9263
Epoch 4/400
18/18 [==============================] - 0s 1ms/step - loss: 0.3253 - accuracy: 0.9263
Epoch 5/400
18/18 [==============================] - 0s 2ms/step - loss: 0.3019 - accuracy: 0.9263
Epoch 6/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2860 - accuracy: 0.9263
Epoch 7/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2749 - accuracy: 0.9263
Epoch 8/400
18/18 [==============================] - 0s 1ms/step - loss: 0.2663 - accuracy: 0.9263
Epoch 9/400
18/18 [==============================] - 0s 1ms/step - loss: 0.2594 - accuracy: 0.9263
Epoch 10/400
18/18 [==============================] - 0s 1ms/step - loss: 0.2536 - accuracy: 0.9263
Epoch 11/

In [ ]:
loss_value, accuracy_value = model.evaluate(test_data, test_labels)
print(f'Our test accuracy was {accuracy_value}')

5/5 [==============================] - 0s 2ms/step - loss: 0.0325 - accuracy: 0.9801
Our test accuracy was 0.9801324605941772


In [ ]:
def predictor(test_data, test_labels, index):
    prediction = model.predict(test_data)
    if np.argmax(prediction[index]) == test_labels[index]:
        print(f'This was correctly predicted to be a \"{test_labels[index]}\"!')
    else:
        print(f'This was incorrectly predicted to be a \"{np.argmax(prediction[index])}\". It was actually a \"{test_labels[index]}\".')
        return(prediction)

In [ ]:
predictor(test_data, test_labels, 149)


This was correctly predicted to be a "1"!
